In [3]:
import numpy as np

# importing Qiskit
from qiskit import IBMQ
from qiskit import QuantumCircuit, transpile
from qiskit.tools import job_monitor
from qiskit.providers.ibmq.managed import IBMQJobManager


provider = IBMQ.load_account()

provider=IBMQ.get_provider(hub='ibm-q-fraunhofer')
backend = provider.get_backend('ibm_lagos')



ibmqfactory.load_account:WARNING:2023-02-14 11:14:14,884: Credentials are already in use. The existing account in the session will be replaced.


In [ ]:
#LAUNCH JOBS ON REAL BACKEND


#Parameters (editable)
step=1 #Trotter step to lauch on real backend
nb_twirls=300 #number of twirls
nb_shots=32000 #number of shots
measurement='ZZZ'
#measurement='XYZ'
randomized_compiling='standard'
#randomized_compiling='crosstalk'
ini_lay=[18,21,23] #qubit on which is performed the simulation


g=0.5
dt=0.2
L=backend.configuration().num_qubits

qc_meas=list([] for _ in range(2*L+8))#Quantum Circuit for measurement calibration used for readout error correction

#protocol 1
for j in range(2):
    qc=QuantumCircuit(1,1)
    if j==1:
        qc.x(0) #initialization in state 1
    qc.measure(0,0)
    for i in range(L):
        qc_meas[2*i+j]=transpile(qc,backend=backend,optimization_level=0,
                                    basis_gates=['cx','x','sx','rz','id'],initial_layout=[i])#measure each qubit separately

#protocol 2
for k in range(8):
    qc=QuantumCircuit(3,3)#initialisation of the 8 differents bit string configurations
    if k//2*2!=k:
        qc.x(0)
    if (k//2)//2*2!=(k//2):
        qc.x(1)
    if k>=4:
        qc.x(2)
    qc.barrier()
    qc.measure(0,0)
    qc.measure(1,1)
    qc.measure(2,2)
    qc_meas[2*L+k]=transpile(qc,backend=backend,optimization_level=0,
                            basis_gates=['cx','x','sx','rz','id'],initial_layout=ini_lay)#measure all the qubits used 

job_manager = IBMQJobManager()      
T=dt*step

PATH_trotter='data/'+measurement+'/T='+str(round(T,1))+'/randomized_compiling/Quantum_Circuit/trotter_'
PATH_estimation='data/ZZZ/T='+str(round(T,1))+'/estimation_circuit/Quantum_Circuit/estimation_'

name=randomized_compiling+'_twirl_g='+str(g)+'_dt='+str(dt)+'_backend='+str(backend)


#launch BCS raw circuit
qc_trot=QuantumCircuit.from_qasm_file(PATH_trotter+'/trotter_g='+str(g)+'_dt='+str(dt)+'_backend='+str(backend)+'.qasm')
job_trot = backend.run(qc_trot,job_name='Trotter_raw_'+measurement+'_'+randomized_compiling+'_T='+str(round(T,1))
                       ,shots=nb_shots)#launch Trotter circuit without any error mitigation protocol

#launch measurement calibration circuit for readout error correction
job_meas = job_manager.run(qc_meas, backend=backend, name='Meas_'+measurement+'_'+randomized_compiling+
                           '_T='+str(round(T,1)),shots=nb_shots)#launch Quantum Circuit for readout error correction

#launch the different randomized version of the Trotter circuit
qc_randomized=list(QuantumCircuit.from_qasm_file(PATH_trotter+name+'_'+str(k)+'.qasm') for k in range(nb_twirls))    

job_randomized = job_manager.run(qc_randomized, backend=backend, name='Trotter_'+measurement+'_'+randomized_compiling+
                                 '_T='+str(round(T,1)),shots=nb_shots)#launch Trotter circuit with randomized compiling

#launch noise estimation circuits
qc_estimation=list(QuantumCircuit.from_qasm_file(PATH_estimation+name+'_'+str(k)+'.qasm') for k in range(nb_twirls))   

job_estimation = job_manager.run(qc_estimation, backend=backend, name='Estimation_'+measurement+'_'+randomized_compiling+
                                 '_T='+str(round(T,1)),shots=nb_shots)#launch estimation circuit with randomized compiling
    